## Import stuff

In [11]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())


2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9951742686123806768
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


In [3]:


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


## Load training data from csv


In [4]:
features_df_orig = pd.read_csv("train_features_fixed.csv")
features_df_normalized = pd.read_csv("train_features_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")


# We can use a different version of the training data for each task
X_orig = np.array(features_df_orig.iloc[:, 2:].values,  dtype=float)
X_orig = X_orig.reshape(227940//12,35*12)
X_normalized = np.array(features_df_normalized.iloc[:, 2:].values,  dtype=float)
X_normalized = X_normalized.reshape(227940//12,35*12)
X_small = np.array(features_df_normalized.iloc[:, 2:].values,  dtype=float)
X_small = X_small[::12]


#use train to validation ratio of 0.8
X_orig_train = X_orig[:int(X_orig.shape[0]*0.8)]
X_orig_valid = X_orig[int(X_orig.shape[0]*0.8):]
X_normalized_train = X_normalized[:int(X_normalized.shape[0]*0.8)]
X_normalized_valid = X_normalized[int(X_normalized.shape[0]*0.8):]
X_small_train = X_small[:int(X_small.shape[0]*0.8)]
X_small_valid = X_small[int(X_small.shape[0]*0.8):]

Y1 = np.array(labels_df.iloc[:, 1:11].values,  dtype=float)
Y1_train = Y1[:int(Y1.shape[0]*0.8)]
Y1_valid = Y1[int(Y1.shape[0]*0.8):]

Y2 = np.array(labels_df.iloc[:, 11].values,  dtype=float)
Y2_train = Y2[:int(Y2.shape[0]*0.8)]
Y2_valid = Y2[int(Y2.shape[0]*0.8):]

Y3 = np.array(labels_df.iloc[:, 12:].values,  dtype=float)
Y3_train = Y3[:int(Y3.shape[0]*0.8)]
Y3_valid = Y3[int(Y3.shape[0]*0.8):]

print("X_orig shape: ", X_orig.shape)
print("X_normalized shape: ", X_normalized.shape)
print("X_small shape: ", X_small.shape)
print("Y1 shape: ", Y1.shape)
print("Y2 shape: ", Y2.shape)
print("Y3 shape: ", Y3.shape)





# print(inputs_dim)
# print(outputs_dim)

# X = tf.cast(X, tf.float32)
# Y = tf.cast(Y, tf.float32)

#print(X.shape)
# random.shuffle(image_ids)
# random.shuffle(diagnostics)

X_orig shape:  (18995, 420)
X_normalized shape:  (18995, 420)
X_small shape:  (18995, 35)
Y1 shape:  (18995, 10)
Y2 shape:  (18995,)
Y3 shape:  (18995, 4)


In [12]:
#over and under sampling

# ros = RandomOverSampler(random_state=0)
ros = ClusterCentroids(random_state=0)
X_small_resampled, Y2_resampled = ros.fit_resample(X_small_train, Y2_train)

print(Y2_resampled[Y2_resampled==1].shape)
print(Y2_resampled[Y2_resampled==0].shape)

(886,)
(886,)


## TASK 1: ORDERING OF MEDICAL TEST

In [ ]:
inputs_dim = X_normalized.shape[1]
outputs_dim = Y1.shape[1]

model1 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='sigmoid'),
  tf.keras.layers.Dense(units=25, activation='sigmoid'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model1.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model1.fit(X_normalized_train, Y1_train, epochs=10) # , verbose=False
print("Finished training the model")

## TASK 1: EVALUATION

In [ ]:
pred = model1.predict(X_normalized_valid)
print(np.var(pred))
for i in range(Y1_valid.shape[1]):
    print("score for col ",i,": " , metrics.roc_auc_score(Y1_valid[:,i], pred[:,i]))



## TASK 2: SEPSIS PREDICTION

In [13]:
kernel = "rbf"
reg = np.power(10., -1.5)
bandwidth = np.power(10., -0.6)
gamma = np.power(10., -bandwidth)
coef0 = 0
tol = 1e-1

model2 = svm.SVC(kernel=kernel,
                         C=np.power(10., -reg),
                         gamma=gamma,
                         coef0=coef0,
                         tol=tol,
                         random_state=10)

                        
                        
model2.fit(X_small_resampled[:10000], Y2_resampled[:10000])

SVC(C=0.929773627269766, coef0=0, gamma=0.5608043282783708, random_state=10,
    tol=0.1)

## TASK 2: EVALUATION

In [14]:
pred = model2.predict(X_small_valid)
print(np.var(pred))
print(pred.shape)
print(Y2_valid.shape)
print(Y2_valid[Y2_valid==1].shape)
print(Y2_valid[Y2_valid==0].shape)

print("score : " , metrics.roc_auc_score(Y2_valid, pred.reshape(-1,1)))

0.17280700557031364
(3799,)
(3799,)
(202,)
(3597,)
score :  0.48137887183213734


## TASK 3: KEYS VITALS SIGNS PREDICTION

In [ ]:
inputs_dim = X_orig.shape[1]
outputs_dim = Y3.shape[1]

model3 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='relu'),
  tf.keras.layers.Dense(units=25, activation='relu'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model3.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model3.fit(X_orig_train, Y3_train, epochs=5) # , verbose=False
print("Finished training the model")

## TASK 3: EVALUATION

In [62]:
pred = model3.predict(X_orig_valid)
for i in range(Y3_valid.shape[1]):
    print("score for col ",i,": " , 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y3_valid[i], pred[i])))



score for col  0 :  0.9781298142088239
score for col  1 :  0.9921901556178777
score for col  2 :  0.9255223387803362
score for col  3 :  0.8847010520610992


## Writing results